In [1]:
import numpy as np
import pandas as pd
import itertools

from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import seaborn as sns

#### Show All Columns

In [ ]:
pd.set_option('display.max_columns', None)

#### Read dataset

In [3]:
train = pd.read_csv("./godaddy-microbusiness-density-forecasting/train.csv")
test = pd.read_csv("./godaddy-microbusiness-density-forecasting/test.csv")
census = pd.read_csv("./godaddy-microbusiness-density-forecasting/census_starter.csv")
submission = pd.read_csv("./godaddy-microbusiness-density-forecasting/sample_submission.csv")

In [5]:
data = pd.concat((train, test))
data = data.sort_values('row_id').reset_index(drop=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147345 entries, 0 to 147344
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 147345 non-null  object 
 1   cfips                  147345 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     147345 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 7.9+ MB


#### Get Month and Year

In [6]:
data['first_day_of_month'] = pd.to_datetime(data.first_day_of_month, format='%Y-%m-%d')
data['year']= data['first_day_of_month'].dt.year-2

In [7]:
data.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year
0,10001_2019-08-01,10001,Kent County,Delaware,2019-08-01,4.995701,6624.0,2017
1,10001_2019-09-01,10001,Kent County,Delaware,2019-09-01,5.031902,6672.0,2017
2,10001_2019-10-01,10001,Kent County,Delaware,2019-10-01,5.098270,6760.0,2017
3,10001_2019-11-01,10001,Kent County,Delaware,2019-11-01,5.124666,6795.0,2017
4,10001_2019-12-01,10001,Kent County,Delaware,2019-12-01,5.086203,6744.0,2017


In [8]:
census['cfips'] = census['cfips'].astype("string")
census.describe()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
count,3142.000000,3142.000000,3142.000000,3141.000000,3141.000000,3142.000000,3142.000000,3142.000000,3141.000000,3141.000000,...,3142.000000,3141.000000,3142.000000,3141.000000,3141.000000,3142.000000,3141.000000,3142.000000,3140.000000,3140.000000
mean,69.920401,72.690866,75.398600,78.543298,80.539096,13.813399,14.005379,14.240452,14.631328,14.851226,...,1.427148,1.382171,1.338956,1.309265,1.272684,49754.163908,51583.175422,53475.909612,55012.443312,58223.339490
std,9.702052,9.255863,8.846665,8.250864,7.889931,5.586649,5.630199,5.689780,5.776940,5.832285,...,0.986985,0.942777,0.940891,0.990342,0.968579,13154.424871,13703.502202,14192.529173,14649.568591,15541.038914
min,24.500000,25.700000,34.800000,33.300000,37.000000,2.400000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,19264.000000,20188.000000,21504.000000,22292.000000,17109.000000
25%,64.200000,67.425000,70.500000,74.100000,76.400000,9.700000,9.900000,10.100000,10.500000,10.600000,...,0.800000,0.800000,0.700000,0.700000,0.600000,41123.000000,42480.000000,44155.000000,45653.000000,48179.500000
50%,70.700000,73.600000,76.450000,79.600000,81.700000,12.800000,13.000000,13.250000,13.600000,13.800000,...,1.300000,1.300000,1.200000,1.200000,1.100000,48065.500000,49888.000000,51757.500000,52842.000000,55907.000000
75%,76.400000,78.800000,81.400000,84.100000,85.900000,16.800000,17.100000,17.300000,17.900000,18.000000,...,1.900000,1.800000,1.800000,1.800000,1.700000,55763.500000,57611.000000,59867.250000,61500.750000,64930.000000
max,94.600000,95.500000,96.000000,97.100000,97.600000,43.700000,48.000000,45.400000,43.000000,43.700000,...,17.400000,11.700000,10.500000,15.200000,15.200000,129588.000000,136268.000000,142299.000000,147111.000000,156821.000000


In [9]:
census[pd.isnull(census["pct_bb_2021"])]

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
92,80.5,79.1,80.4,NaN,NaN,2261,23.1,19.0,16.5,NaN,...,3.3,3.9,5.3,NaN,NaN,86019,82306.0,79867,NaN,NaN


In [10]:
cleaned_census_dataset = census.bfill()

In [11]:
transaction_col = ['cfips','year','pct_bb','pct_college','pct_foreign_born','pct_it_workers','median_hh_inc']
transaction_df = pd.DataFrame(columns=transaction_col)

In [12]:
cleaned_census_dict = cleaned_census_dataset.to_dict(orient='index')

for index in np.arange(len(cleaned_census_dataset)):
    current_row = cleaned_census_dict[index]
    for year in [2017,2018,2019,2020,2021]:
        row_dict = dict.fromkeys(transaction_col)
        row_dict['cfips']=current_row['cfips']
        row_dict['year']=year
        row_dict['pct_bb']=current_row['pct_bb_' + (str)(year)]
        row_dict['pct_college']=current_row['pct_college_' + (str)(year)]
        row_dict['pct_foreign_born']=current_row['pct_foreign_born_' + (str)(year)]
        row_dict['pct_it_workers']=current_row['pct_it_workers_' + (str)(year)]
        row_dict['median_hh_inc']=current_row['median_hh_inc_' + (str)(year)]
        transaction_df = transaction_df.append(row_dict, ignore_index=True)
        
transaction_df.head()       

,cfips,year,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc
0,1001,2017,76.6,14.5,2.1,1.3,55317
1,1001,2018,78.9,15.9,2.0,1.1,58786
2,1001,2019,80.6,16.1,2.3,0.7,58731
3,1001,2020,82.7,16.7,2.3,0.6,57982
4,1001,2021,85.5,16.4,2.1,1.1,62660


In [13]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15710 entries, 0 to 15709
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cfips             15710 non-null  object 
 1   year              15710 non-null  object 
 2   pct_bb            15710 non-null  float64
 3   pct_college       15710 non-null  float64
 4   pct_foreign_born  15710 non-null  float64
 5   pct_it_workers    15710 non-null  float64
 6   median_hh_inc     15710 non-null  object 
dtypes: float64(4), object(3)
memory usage: 859.3+ KB


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147345 entries, 0 to 147344
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 147345 non-null  object        
 1   cfips                  147345 non-null  int64         
 2   county                 122265 non-null  object        
 3   state                  122265 non-null  object        
 4   first_day_of_month     147345 non-null  datetime64[ns]
 5   microbusiness_density  122265 non-null  float64       
 6   active                 122265 non-null  float64       
 7   year                   147345 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 9.0+ MB


In [15]:
data['cfips'] = data['cfips'].astype("string")
data['county'] = data.groupby('cfips')['county'].ffill()
data['state'] = data.groupby('cfips')['state'].ffill()

In [16]:
# Join table
data_merged = pd.merge(transaction_df, data, on=['cfips','year'])

In [17]:
data_merged['year'].unique()

array([2017, 2018, 2019, 2020, 2021], dtype=object)

In [18]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147345 entries, 0 to 147344
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   cfips                  147345 non-null  object        
 1   year                   147345 non-null  object        
 2   pct_bb                 147345 non-null  float64       
 3   pct_college            147345 non-null  float64       
 4   pct_foreign_born       147345 non-null  float64       
 5   pct_it_workers         147345 non-null  float64       
 6   median_hh_inc          147345 non-null  object        
 7   row_id                 147345 non-null  object        
 8   county                 147345 non-null  object        
 9   state                  147345 non-null  object        
 10  first_day_of_month     147345 non-null  datetime64[ns]
 11  microbusiness_density  122265 non-null  float64       
 12  active                 122265 non-null  floa

In [19]:
data_merged.isnull().sum()

cfips                        0
year                         0
pct_bb                       0
pct_college                  0
pct_foreign_born             0
pct_it_workers               0
median_hh_inc                0
row_id                       0
county                       0
state                        0
first_day_of_month           0
microbusiness_density    25080
active                   25080
dtype: int64

In [20]:
train = data_merged.dropna()
test = data_merged[data_merged['microbusiness_density'].isna()]

In [21]:
train_predata_merged =train[['pct_bb','pct_college','pct_foreign_born','pct_it_workers','median_hh_inc','county','state']]
test_predata_merged =test[['pct_bb','pct_college','pct_foreign_born','pct_it_workers','median_hh_inc','county','state']]

In [23]:
X = pd.get_dummies(train_predata_merged ,columns=['county','state'],drop_first=True)
Xtest = pd.get_dummies(test_predata_merged,columns=['county','state'],drop_first=True)

In [24]:
from sklearn.compose import ColumnTransformer

ct_model = ColumnTransformer([
        ('somename', StandardScaler(), ['pct_bb','pct_college','pct_foreign_born','pct_it_workers','median_hh_inc'])
    ], remainder='passthrough')

X=ct_model.fit_transform(X)
Xtest=ct_model.fit_transform(Xtest)

In [25]:
X = np.asarray(X).astype('float32')

In [26]:
X

array([[ 0.20413265,  0.04921869, -0.4631215 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20413265,  0.04921869, -0.4631215 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20413265,  0.04921869, -0.4631215 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.5348187 , -0.2673601 , -0.42810196, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.5348187 , -0.2673601 , -0.42810196, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.5348187 , -0.2673601 , -0.42810196, ...,  0.        ,
         0.        ,  1.        ]], dtype=float32)

In [27]:
y = train['microbusiness_density']

In [29]:
from tensorflow.keras.metrics import MeanSquaredError,MeanAbsoluteError

model = Sequential()
model.add(Dense(100, input_shape=(1925,), kernel_initializer='normal', activation='relu'))
model.add(Dense(100, input_shape=(1925,) ,activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear', kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=[MeanSquaredError(),MeanAbsoluteError()])

In [30]:
model.fit(X, y, epochs=20, batch_size=10)

Epoch 1/20
12227/12227 [==============================] - 55s 4ms/step - loss: 9.5486 - mean_squared_error: 9.5486 - mean_absolute_error: 0.7841
Epoch 2/20
12227/12227 [==============================] - 56s 5ms/step - loss: 6.5525 - mean_squared_error: 6.5525 - mean_absolute_error: 0.4963
Epoch 3/20
12227/12227 [==============================] - 50s 4ms/step - loss: 6.3424 - mean_squared_error: 6.3424 - mean_absolute_error: 0.4467
Epoch 4/20
12227/12227 [==============================] - 52s 4ms/step - loss: 5.6092 - mean_squared_error: 5.6092 - mean_absolute_error: 0.4175
Epoch 5/20
12227/12227 [==============================] - 52s 4ms/step - loss: 5.3244 - mean_squared_error: 5.3244 - mean_absolute_error: 0.3730
Epoch 6/20
12227/12227 [==============================] - 47s 4ms/step - loss: 5.3556 - mean_squared_error: 5.3556 - mean_absolute_error: 0.3507
Epoch 7/20
12227/12227 [==============================] - 50s 4ms/step - loss: 4.5898 - mean_squared_error: 4.5898 - mean_absolute

In [31]:
y_predicted= model.predict(Xtest)
y_predicted = list(itertools.chain(*y_predicted))

y_predicted

784/784 [==============================] - 2s 2ms/step


[3.1701229,
 3.1701229,
 3.2140782,
 3.2140782,
 3.2140782,
 3.2140782,
 3.2140782,
 3.2140782,
 7.481602,
 7.481602,
 7.728435,
 7.728435,
 7.728435,
 7.728435,
 7.728435,
 7.728435,
 1.1842664,
 1.1842664,
 1.1024363,
 1.1024363,
 1.1024363,
 1.1024363,
 1.1024363,
 1.1024363,
 1.3208836,
 1.3208836,
 1.5402099,
 1.5402099,
 1.5402099,
 1.5402099,
 1.54021,
 1.54021,
 1.758159,
 1.758159,
 1.7405294,
 1.7405294,
 1.7405294,
 1.7405294,
 1.7405294,
 1.7405294,
 0.86928093,
 0.86928093,
 0.90603656,
 0.90603656,
 0.90603656,
 0.90603656,
 0.90603656,
 0.90603656,
 1.5884479,
 1.5884479,
 1.751609,
 1.751609,
 1.751609,
 1.751609,
 1.751609,
 1.751609,
 2.5059433,
 2.5059433,
 2.519824,
 2.519824,
 2.519824,
 2.519824,
 2.519824,
 2.519824,
 1.455236,
 1.455236,
 1.1720613,
 1.1720613,
 1.1720613,
 1.1720613,
 1.1720613,
 1.1720613,
 1.2148769,
 1.2148769,
 1.2835724,
 1.2835724,
 1.2835724,
 1.2835724,
 1.2835724,
 1.2835724,
 1.5755188,
 1.5755188,
 1.7301657,
 1.7301657,
 1.7301657,


In [32]:
submission = submission[['row_id']]

In [33]:
submission['microbusiness_density'] = y_predicted

In [34]:
submission.head()

,row_id,microbusiness_density
0,1001_2022-11-01,3.170123
1,1003_2022-11-01,3.170123
2,1005_2022-11-01,3.214078
3,1007_2022-11-01,3.214078
4,1009_2022-11-01,3.214078


In [35]:
submission.to_csv('submission_tensorflow.csv', index=False)